아래는 백화점 고객의 1년 간 구매 데이터이다.

(가) 제공 데이터 목록
- y_train.csv : 고객의 성별 데이터 (학습용), CSV 형식의 파일
- X_train.csv, X_test.csv : 고객의 상품구매 속성 (학습용 및 평가용), CSV 형식의 파일

(나) 데이터 형식 및 내용
- y_train.csv (3,500명 데이터)
- X_train.csv (3,500명 데이터), X_test.csv (2,482명 데이터)

고객 3,500명에 대한 학습용 데이터(y_train.csv, X_train.csv)를 이용하여 성별예측 모형을 만든 후, 이를 평가용 데이터(X_test.csv)에 적용하여 얻은 2,482명 고객의 성별 예측값(남자일 확률)을 다음과 같은 형식의 CSV 파일로 생성하시오.(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)

custid,gender
3500,0.267
3501,0.578
3502,0.885
․ ․ ․


성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, Feature Engineering, 분류 알고리즘 사용, 초매개변수 최적화, 모형 앙상블 등이 수반되어야 한다

In [1]:
# 환불금액 원핫인코딩
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [2]:
X = pd.read_csv('./dataset/X_train.csv',encoding='cp949') 
y = pd.read_csv('./dataset/y_train.csv',encoding='cp949')
test = pd.read_csv('./dataset/X_test.csv',encoding='cp949')
# X_train.drop('cust_id',axis=1,inplace=True)
# y_train.drop('cust_id',axis=1,inplace=True)
# X_test.drop('cust_id',axis=1,inplace=True)

In [69]:
X

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85
...,...,...,...,...,...,...,...,...,...,...
3495,3495,3175200,3042900,NaN,골프,본 점,1,2.000000,1.000000,0
3496,3496,29628600,7200000,6049600.0,시티웨어,부산본점,8,1.625000,0.461538,40
3497,3497,75000,75000,NaN,주방용품,창원점,1,1.000000,0.000000,0
3498,3498,1875000,1000000,NaN,화장품,본 점,2,1.000000,0.000000,39


In [70]:
y

,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0
...,...,...
3495,3495,1
3496,3496,1
3497,3497,0
3498,3498,0


In [71]:
test

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,3500,70900400,22000000,4050000.0,골프,부산본점,13,1.461538,0.789474,26
1,3501,310533100,38558000,48034700.0,농산물,잠실점,90,2.433333,0.369863,3
2,3502,305264140,14825000,30521000.0,가공식품,본 점,101,14.623762,0.083277,3
3,3503,7594080,5225000,NaN,주방용품,부산본점,5,2.000000,0.000000,47
4,3504,1795790,1411200,NaN,수산품,청량리점,3,2.666667,0.125000,8
...,...,...,...,...,...,...,...,...,...,...
2477,5977,82581500,23976000,NaN,골프,부산본점,8,1.750000,0.642857,40
2478,5978,480000,480000,NaN,섬유잡화,광주점,1,1.000000,0.000000,0
2479,5979,260003790,25750000,NaN,남성 캐주얼,본 점,19,3.736842,0.915493,18
2480,5980,88991520,18120000,NaN,육류,본 점,5,3.600000,0.444444,60


In [72]:
# 주구매상품 42종
print(len(X.주구매상품.value_counts()))
print(len(test.주구매상품.value_counts()))

42
41


In [84]:
productx = X.주구매상품.unique()
productt = test.주구매상품.unique()
print(productx)
print(productt)
set(productx) - set(productt)

['기타' '스포츠' '남성 캐주얼' '보석' '디자이너' '시티웨어' '명품' '농산물' '화장품' '골프' '구두' '가공식품'
 '수산품' '아동' '차/커피' '캐주얼' '섬유잡화' '육류' '축산가공' '젓갈/반찬' '액세서리' '피혁잡화' '일용잡화'
 '주방가전' '주방용품' '건강식품' '가구' '주류' '모피/피혁' '남성 트랜디' '셔츠' '남성정장' '생활잡화'
 '트래디셔널' '란제리/내의' '커리어' '침구/수예' '대형가전' '통신/컴퓨터' '식기' '소형가전' '악기']
['골프' '농산물' '가공식품' '주방용품' '수산품' '화장품' '기타' '스포츠' '디자이너' '시티웨어' '구두' '캐주얼'
 '명품' '건강식품' '남성정장' '커리어' '남성 캐주얼' '축산가공' '식기' '피혁잡화' '모피/피혁' '섬유잡화'
 '트래디셔널' '차/커피' '육류' '가구' '아동' '셔츠' '액세서리' '젓갈/반찬' '대형가전' '일용잡화' '통신/컴퓨터'
 '생활잡화' '주방가전' '란제리/내의' '남성 트랜디' '보석' '주류' '침구/수예' '악기']


{'소형가전'}

In [87]:
X[X.주구매상품=='소형가전']

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
1521,1521,178000,178000,0.0,소형가전,본 점,1,1.0,1.0,0
2035,2035,260000,260000,0.0,소형가전,잠실점,1,1.0,0.0,0


In [3]:
nx = X.drop([1521,2035]).reset_index(drop=True)
ny = y.drop([1521,2035]).reset_index(drop=True)

In [73]:
# 주구매지점 24점
print(len(X.주구매지점.value_counts()))
print(len(test.주구매지점.value_counts()))

24
24


In [74]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB


In [75]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2482 entries, 0 to 2481
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  2482 non-null   int64  
 1   총구매액     2482 non-null   int64  
 2   최대구매액    2482 non-null   int64  
 3   환불금액     871 non-null    float64
 4   주구매상품    2482 non-null   object 
 5   주구매지점    2482 non-null   object 
 6   내점일수     2482 non-null   int64  
 7   내점당구매건수  2482 non-null   float64
 8   주말방문비율   2482 non-null   float64
 9   구매주기     2482 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 194.0+ KB


In [99]:
# 상품, 지점 인코딩
le = LabelEncoder()
col=['주구매상품','주구매지점']
for i in col:    
    nx[i] = le.fit_transform(nx[i])

In [100]:
col=['주구매상품','주구매지점']
for i in col:
    test[i] = le.fit_transform(test[i])

In [4]:
# 환불금액 결측치 처리
nx.loc[nx.환불금액==0]
nx.loc[nx.환불금액.isnull(),'환불금액']=0
test.loc[test.환불금액.isnull(),'환불금액']=0

In [5]:
# 환불금액 존재1 , 존재x 0 구분
nx.loc[nx.환불금액!=0,'환불금액']='y'
nx.loc[nx.환불금액==0,'환불금액']='n'
test.loc[test.환불금액!=0,'환불금액']='y'
test.loc[test.환불금액==0,'환불금액']='n'

In [6]:
nx.drop('cust_id',axis=1,inplace=True)
test1 = test.drop('cust_id',axis=1)

In [7]:
ny = ny.gender

In [8]:
nx.head(3)

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,11264000,y,기타,강남점,19,3.894737,0.527027,17
1,2136000,2136000,y,스포츠,잠실점,2,1.500000,0.000000,1
2,3197000,1639000,n,남성 캐주얼,관악점,2,2.000000,0.000000,1


In [9]:
nx = pd.get_dummies(nx)
test1 = pd.get_dummies(test1)

In [10]:
test.head()

,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기,주구매상품_가공식품,주구매상품_가구,주구매상품_건강식품,...,주구매지점_안양점,주구매지점_영등포점,주구매지점_울산점,주구매지점_인천점,주구매지점_일산점,주구매지점_잠실점,주구매지점_전주점,주구매지점_창원점,주구매지점_청량리점,주구매지점_포항점
0,70900400,22000000,4050000.0,13,1.461538,0.789474,26,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,310533100,38558000,48034700.0,90,2.433333,0.369863,3,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,305264140,14825000,30521000.0,101,14.623762,0.083277,3,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7594080,5225000,0.0,5,2.000000,0.000000,47,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1795790,1411200,0.0,3,2.666667,0.125000,8,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
from sklearn.preprocessing import StandardScaler
# 트레인 기준으로 fit, 테스트 데이터에 그 기준을 적용
sc = StandardScaler()
sc.fit(nx)
nx = sc.transform(nx)
test1 = sc.transform(test1)

In [12]:
nx

array([[-0.14487242, -0.26290806, -0.04764965, ..., -0.03383518,
        -0.15876132, -0.0561656 ],
       [-0.54940267, -0.54821566, -0.26554866, ..., -0.03383518,
        -0.15876132, -0.0561656 ],
       [-0.54291397, -0.56375005, -0.27551356, ..., -0.03383518,
        -0.15876132, -0.0561656 ],
       ...,
       [-0.562007  , -0.61263491, -0.27551356, ..., 29.55503341,
        -0.15876132, -0.0561656 ],
       [-0.55099885, -0.58372283, -0.27551356, ..., -0.03383518,
        -0.15876132, -0.0561656 ],
       [ 1.04656862,  0.46748938, -0.07711252, ..., -0.03383518,
        -0.15876132, -0.0561656 ]])

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(nx, ny, test_size = 0.3, random_state = 121, stratify = ny)


In [12]:
lr = LogisticRegression().fit(X_train, y_train)
probs_lr = lr.predict_proba(X_test)[:,1]
print('Logistic Regression:', roc_auc_score(y_test, probs_lr))
result_lr = roc_auc_score(y_test, probs_lr)

dt = DecisionTreeClassifier().fit(X_train, y_train)
probs_dt = dt.predict_proba(X_test)[:,1]
print('Decision Tree:', roc_auc_score(y_test, probs_dt))
result_dt = roc_auc_score(y_test, probs_dt)

rf = RandomForestClassifier().fit(X_train, y_train)
probs_rf = rf.predict_proba(X_test)[:,1]
print('Random Forest Classifier:', roc_auc_score(y_test, probs_rf))
result_rf = roc_auc_score(y_test, probs_rf)

Logistic Regression: 0.6584636196733984
Decision Tree: 0.5375688472316166
Random Forest Classifier: 0.6387245144458402


In [14]:
parameters = {'max_depth':[10,12,15,18,20],
             'min_samples_split':[2,3,5,7], 'min_samples_leaf':[5,8,12,15]}
# parameters = {'max_depth':[2,3,5,10],
#              'min_samples_split':[2,3,5], }

grid_dclf = GridSearchCV(dt, param_grid=parameters , scoring='roc_auc' , cv=5)
grid_dclf.fit(X_train , y_train)

grid_rclf = GridSearchCV(rf, param_grid=parameters , scoring='roc_auc' , cv=5)
grid_rclf.fit(X_train,y_train)
# print('GridSearchCV 최적 하이퍼 파라미터 :',grid_dclf.best_params_)
# print('GridSearchCV 최고 roc_auc스코어: {0:.4f}'.format(grid_dclf.best_score_))
best_dclf = grid_dclf.best_estimator_
best_rclf = grid_rclf.best_estimator_

prob_d = best_dclf.predict_proba(X_test)[:,1]
prob_r = best_rclf.predict_proba(X_test)[:,1]
roc_auc_d = roc_auc_score(y_test,prob_d)
roc_auc_r = roc_auc_score(y_test,prob_r)
print(f'dt : {roc_auc_d}')
print(f'rf : {roc_auc_r}')


dt : 0.5985621799207654
rf : 0.6530312107449996


In [16]:
prob_d = best_dclf.predict_proba(X_test)[:,1]
prob_d

array([0.32862191, 0.36098131, 0.55674232, ..., 0.18035714, 0.55674232,
       0.55674232])

In [17]:
prob_r = best_rclf.predict_proba(X_test)[:,1]
prob_r

array([0.25323967, 0.43458384, 0.62732828, ..., 0.22788252, 0.46587895,
       0.53837211])

In [15]:
prob = lr.predict_proba(test1)[:,1]
prob

array([0.48138221, 0.13583856, 0.12883406, ..., 0.81995245, 0.47822031,
       0.49495362])

In [16]:
result = pd.concat([test.cust_id, pd.DataFrame(prob)], axis = 1)
result

,cust_id,0
0,3500,0.481382
1,3501,0.135839
2,3502,0.128834
3,3503,0.311961
4,3504,0.477531
...,...,...
2477,5977,0.649629
2478,5978,0.618988
2479,5979,0.819952
2480,5980,0.478220


In [18]:
X_train, X_test, y_train, y_test = train_test_split(nx, ny, test_size = 0.2, random_state = 121, stratify = ny)

lr = LogisticRegression().fit(X_train, y_train)
probs_lr = lr.predict_proba(X_test)[:,1]
print('Logistic Regression:', roc_auc_score(y_test, probs_lr))
result_lr = roc_auc_score(y_test, probs_lr)

dt = DecisionTreeClassifier().fit(X_train, y_train)
probs_dt = dt.predict_proba(X_test)[:,1]
print('Decision Tree:', roc_auc_score(y_test, probs_dt))
result_dt = roc_auc_score(y_test, probs_dt)

rf = RandomForestClassifier().fit(X_train, y_train)
probs_rf = rf.predict_proba(X_test)[:,1]
print('Random Forest Classifier:', roc_auc_score(y_test, probs_rf))
result_rf = roc_auc_score(y_test, probs_rf)

Logistic Regression: 0.64571786550191
Decision Tree: 0.537130974236716
Random Forest Classifier: 0.6442561188887246


In [ ]:
parameters = {'max_depth':[2,3,5,8,10],
             'min_samples_split':[2,3,5,7],}
# parameters = {'max_depth':[2,3,5,10],
#              'min_samples_split':[2,3,5], }

grid_dclf = GridSearchCV(dt, param_grid=parameters , scoring='roc_auc' , cv=5)
grid_dclf.fit(X_train , y_train)

grid_rclf = GridSearchCV(rf, param_grid=parameters , scoring='roc_auc' , cv=5)
grid_rclf.fit(X_train,y_train)
# print('GridSearchCV 최적 하이퍼 파라미터 :',grid_dclf.best_params_)
# print('GridSearchCV 최고 roc_auc스코어: {0:.4f}'.format(grid_dclf.best_score_))
best_dclf = grid_dclf.best_estimator_
best_rclf = grid_rclf.best_estimator_

prob_d = best_dclf.predict_proba(X_test)[:,1]
prob_r = best_rclf.predict_proba(X_test)[:,1]
roc_auc_d = roc_auc_score(y_test,prob_d)
roc_auc_r = roc_auc_score(y_test,prob_r)
print(f'dt : {roc_auc_d}')
print(f'rf : {roc_auc_r}')
